# Base

In [38]:
"""
This module contains the data_structures used in py_search. In particular, it
contains the the :class:`Problem` class, which is used to represent the
different search problems, and the :class:`AnnotatedProblem` class, which wraps
around a specific problem and keeps track of the number of core method calls.

At a lower level this module also contains the :class:`Node` class, which is
used to represent a node in the search space.

Finally, the module contains the :class:`Fringe` class, and its instantiations
(:class:`FIFOQueue`, :class:`LIFOQueue`, and :class:`PrioritySet`). A Fringe is
used to structure the way a search space is explored.
"""
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import division

from collections import deque
from random import choice
from bisect import insort


class Problem(object):
    """
    The basic problem to solve. The main functions that must be defined include
    successors and goal_test. Some search techniques also require the
    random_successor and predecessors methods to be implemented.
    """

    def __init__(self, initial, goal=None, initial_cost=0, extra=None):
        self.initial = Node(initial, None, None, initial_cost, extra=extra)
        self.goal = GoalNode(goal)

    def node_value(self, node):
        """
        Returns the the value of the current node. This is the value being
        minimized by the search. By default the cost is used, but this
        function can be overloaded to include a heuristic.
        """
        return node.cost()

    def predecessors(self, node):
        """
        An iterator that yields all of the predecessors of the current goal.
        """
        raise NotImplementedError("No predecessors function implemented")

    def successors(self, node):
        """
        An iterator that yields all of the successors of the current node.
        """
        raise NotImplementedError("No successors function implemented")

    def random_successor(self, node):
        """
        This method should return a single successor node. This is used
        by some of the search techniques. By default, this just computes all of
        the successors and randomly samples one. This default approach is not
        very efficient, but this funciton can be overridden to generate a
        single successor more efficiently.
        """
        return choice([s for s in self.successors(node)])

    def random_node(self):
        """
        This method returns a random node in the search space. This
        is used by some of the local search / optimization techniques to
        randomly restart search.
        """
        raise NotImplementedError("No random node implemented!")

    def goal_test(self, state_node, goal_node=None):
        """
        Returns true if a goal state is found. This is typically not used by
        the local search / optimization techniques, but some of them use the
        goal test to determine if the search should terminate early. By
        default, this checks if the state equals the goal.
        """
        if goal_node is None:
            goal_node = self.goal
        return state_node == goal_node

class AnnotatedProblem(Problem):
    """
    A Problem class that wraps around another Problem and keeps stats on nodes
    expanded and goal tests performed.
    """

    def __init__(self, problem):
        self.problem = problem
        self.initial = problem.initial
        self.goal = problem.goal
        self.nodes_expanded = 0
        self.goal_tests = 0
        self.nodes_evaluated = 0

    def random_successor(self, node):
        """
        A wrapper for the random_successor method that keeps track of the
        number of nodes expanded.
        """
        self.nodes_expanded += 1
        return self.problem.random_successor(node)

    def random_node(self):
        """
        A wrapper for the random_node method.
        """
        return self.problem.random_node()

    def node_value(self, node):
        """
        A wraper for the node value method that keeps track of the number of
        times a node value was calculated.
        """
        self.nodes_evaluated += 1
        return self.problem.node_value(node)

    def predecessors(self, node):
        """
        A wrapper for the predecessors method that keeps track of the number of
        nodes expanded.
        """
        for s in self.problem.predecessors(node):
            self.nodes_expanded += 1
            yield s

    def successors(self, node):
        """
        A wrapper for the successor method that keeps track of the number of
        nodes expanded.
        """
        for s in self.problem.successors(node):
            self.nodes_expanded += 1
            yield s

    def goal_test(self, state_node, goal_node=None):
        """
        A wrapper for the goal_test method that keeps track of the number of
        goal_tests performed.
        """
        self.goal_tests += 1
        return self.problem.goal_test(state_node, goal_node)
    
class Node(object):
    """
    A class to represent a node in the search. This node stores state
    information, path to the state, cost of the node, depth of the node, and
    any extra information.

    :param state: the state at this node
    :type state: object for tree search and hashable object for graph search
    :param parent: the node from which the current node was generated
    :type parent: :class:`Node`
    :param action: the action performed to transition from parent to current.
    :type action: typically a string, but can be any object
    :param cost: the cost of reaching the current node
    :type cost: float
    :param extra: extra information to store in this node, typically used to
                  store non-hashable information about the state.
    :type extra: object
    """

    def __init__(self, state, parent=None, action=None, node_cost=0,
                 extra=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.node_cost = node_cost
        self.extra = extra

        if parent is None:
            self.node_depth = 0
        else:
            self.node_depth = parent.depth() + 1

    def depth(self):
        """
        Returns the depth of the current node.
        """
        return self.node_depth

    def cost(self):
        """
        Returns the cost of the current node.
        """
        return self.node_cost

    def path(self):
        """
        Returns a path (tuple of actions) from the initial to current node.
        """
        actions = []
        current = self
        while current.parent:
            actions.append(current.action)
            current = current.parent
        actions.reverse()
        return tuple(actions)

    def __str__(self):
#         return "State: %s, Extra: %s" % (self.state, self.extra)
        return "State: %s, Cost: %s" % (self.state, self.node_cost)

    def __repr__(self):
        return "Node(%s, Cost(%s))" % (repr(self.state), repr(self.node_cost))

    def __hash__(self):
        return hash(self.state)

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __ne__(self, other):
        return not self.__eq__(other)

    def __lt__(self, other):
        return self.cost() < other.cost()


class GoalNode(Node):
    """
    Used to represent goals in the backwards portion of the search.
    """

    def __repr__(self):
        return "GoalNode(%s)" % repr(self.state)

    def path(self):
        """
        Returns a path (tuple of actions) from the initial to current node.

        Similar to Node's path function, but returns the path in the opposite
        order because the goal nodes branch out from the goal (not the start
        state).
        """
        actions = []
        current = self
        while current.parent:
            actions.append(current.action)
            current = current.parent
        return tuple(actions)


class SolutionNode(object):
    """
    A Node class that joins a state (:class:`Node`) and a goal
    (:class:`GoalNode`) in bidirectional search, so that it can be returned and
    the used like other :class:`Node`. In particular it provides an isomorphic
    interface for querying depth, cost, and path.

    The state, parent, action, node_cost, and extra attributes have been
    removed because they are not well defined for a join. The key issue here is
    that the state and goal nodes might not be specified in the same terms. For
    example, goals may be partial states and goal_test might return True when
    the state_node satisfies the goal_node (not when they are strictly equal).

    Thus, to generate the actual state represented by the solution node, the
    returned path needs to be executed from the initial state, which is outside
    the scope of this library since it has no knowledge of how to execute paths
    (it just generates them using the user specified successor/predecessor
    functions).
    """

    def __init__(self, state, goal):
        self.state_node = state
        self.goal_node = goal

    def depth(self):
        return self.state_node.depth() + self.goal_node.depth()

    def cost(self):
        return self.state_node.cost() + self.goal_node.cost()

    def path(self):
        return self.state_node.path() + self.goal_node.path()

    def __str__(self):
        return "StateNode={%s}, GoalNode={%s}" % (self.state_node,
                                                  self.goal_node)

    def __repr__(self):
        return "SolutionNode(%s, %s)" % (repr(self.state_node),
                                         repr(self.goal_node))

    def __hash__(self):
        return hash((self.state_node.state, self.goal_node.state))

    def __eq__(self, other):
        return (isinstance(other, SolutionNode) and
                self.state_node == other.state_node and
                self.goal_node == other.goal_node)

    def __ne__(self, other):
        return not self.__eq__(other)


class Fringe(object):
    """
    A template for a fringe class. Used to control the strategy of different
    search approaches.
    """

    def push(self, node):
        """
        Adds one node to the collection.
        """
        raise NotImplementedError("No push method")

    def extend(self, nodes):
        """
        Given an iterator (`nodes`) adds all the nodes to the collection.
        """
        for n in nodes:
            self.push(n)

    def pop(self):
        """
        Pops a node off the collection.
        """
        raise NotImplementedError("No pop method")

    def __len__(self):
        """
        Returns the length of the fringe.
        """
        raise NotImplementedError("No __len__ method")

    def __iter__(self):
        """
        Returns iterator that yields the elements in the order they would be
        popped.
        """
        raise NotImplementedError("No __iter__ method")


class FIFOQueue(Fringe):
    """
    A first-in-first-out queue. Used to get breadth first search behavior.

    >>> fifo = FIFOQueue()
    >>> fifo.push(0)
    >>> fifo.push(1)
    >>> fifo.push(2)
    >>> list(fifo)
    [0, 1, 2]
    >>> fifo.remove(2)
    >>> print(fifo.pop())
    0
    >>> print(fifo.pop())
    1
    """

    def __init__(self):
        self.nodes = deque()

    def push(self, node):
        self.nodes.append(node)

    def remove(self, node):
        for i in range(self.nodes.count(node)):
            self.nodes.remove(node)

    def pop(self):
        return self.nodes.popleft()

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        return iter(self.nodes)


class LIFOQueue(FIFOQueue):
    """
    A last-in-first-out queue. Used to get depth first search behavior.

    >>> lifo = LIFOQueue()
    >>> lifo.push(0)
    >>> lifo.push(1)
    >>> lifo.push(2)
    >>> list(lifo)
    [2, 1, 0]
    >>> print(lifo.pop())
    2
    >>> print(lifo.pop())
    1
    >>> print(lifo.pop())
    0
    """

    def pop(self):
        return self.nodes.pop()

    def __iter__(self):
        return reversed(self.nodes)


class PriorityQueue(Fringe):
    """
    A priority queue that sorts elements by their value. Always returns the
    minimum value item.  A :class:`PriorityQueue` accepts a node_value
    function, a cost_limit (nodes with a value greater than this limit will not
    be added) and a max_length parameter. If adding an item ever causes the
    size to exceed the max_length then the worst nodes are removed until the
    list is equal to max_length.

    >>> pq = PriorityQueue(node_value=lambda x: x, max_length=3)
    >>> pq.push(6)
    >>> pq.push(0)
    >>> pq.push(2)
    >>> pq.push(6)
    >>> pq.push(7)
    >>> len(pq)
    3
    >>> list(pq)
    [0, 2, 6]
    >>> pq.update_cost_limit(5)
    >>> len(pq)
    2
    >>> pq.peek()
    0
    >>> pq.peek_value()
    0
    >>> print(pq.pop())
    0
    >>> pq.peek()
    2
    >>> pq.peek_value()
    2
    >>> print(pq.pop())
    2
    >>> len(pq)
    0

    :param node_value: The node evaluation function (defaults to
        ``lambda x: x.cost()``)
    :type node_value: a function with one parameter for node
    :param cost_limit: the maximum value for elements in the set, if an item
        exceeds this limit then it will not be added (defaults to
        ``float('inf'))``
    :type cost_limit: float
    :param max_length: The maximum length of the list (defaults to
        ``float('inf')``
    :type max_length: int or ``float('inf')``
    """

    def __init__(self, node_value=lambda x: x, cost_limit=float('inf'),
                 max_length=float('inf')):
        self.nodes = []
        self.max_length = max_length
        self.cost_limit = cost_limit
        self.node_value = node_value

    def clear(self):
        """
        Empties the list.
        """
        self.nodes = []

    def peek(self):
        """
        Returns the best node.
        """
        return self.nodes[-1][1]

    def peek_value(self):
        """
        Returns the value of the best node.
        """
        return -self.nodes[-1][0]

    def update_cost_limit(self, cost_limit):
        """
        Updates the cost limit and removes any nodes that violate the new
        limit.
        """
        self.cost_limit = cost_limit
        for i in range(len(self.nodes)):
            if self.nodes[i][0] >= -self.cost_limit:
                self.nodes = self.nodes[i:]
                break

    def push(self, node):
        """
        Push a node into the priority queue. If the node exceeds the cost limit
        then it is not added. If the max_length is exceeded by
        adding the node, then the worst node is discarded from the set.
        """
        value = self.node_value(node)

        if value > self.cost_limit:
            return

        insort(self.nodes, (-value, node))

        if len(self.nodes) > self.max_length:
            val, node = self.nodes.pop(0)

    def pop(self):
        """
        Pop the best value from the priority queue.
        """
        val, node = self.nodes.pop()
        return node

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        for v, n in reversed(self.nodes):
            yield n

# Oprimisation algorithms

In [39]:
"""
This module contains the local search / optimization techniques. Instead of
trying to find a goal state, these algorithms try to find the lowest cost
state.
"""
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import division

from math import exp
from math import log
from random import random

# from py_search.base import PriorityQueue
# from py_search.base import SolutionNode


def branch_and_bound(problem, graph=True, depth_limit=float('inf')):
    """
    An exhaustive optimization technique that is guranteed to give the best
    solution. In general the algorithm starts with some (potentially
    non-optimal) solution. Then it uses the cost of the current best solution
    to prune branches of the search that do not have any chance of being better
    than this solution (i.e., that have a node_value > current best cost).

    In this implementation, node_value should provide an admissible lower bound
    on the cost of solutions reachable from the provided node. If node_value is
    inadmissible, then optimality guarantees are lost.

    Also, if the search space is infinite and/or the node_value function
    provides too little guidance (e.g., node_value = float('-inf')), then
    the search might never terminate. To counter this, a depth_limit can be
    provided that stops expanding nodes after the provided depth. This will
    ensure the search is finite and guaranteed to terminate.

    Finally, the problem.goal_test function can be used to terminate search
    early if a good enough solution has been found. If goal_test(node) return
    True, then search is immediately terminated and the node is returned.

    Note, the current implementation uses best-first search via a priority
    queue data structure.

    :param problem: The problem to solve.
    :type problem: :class:`py_search.base.Problem`
    :param graph: Whether to use graph search (no duplicates) or tree search
                  (duplicates).
    :type graph: Boolean
    """
    b = None
    bv = float('inf')

    fringe = PriorityQueue(node_value=problem.node_value)
    fringe.push(problem.initial)

    if graph:
        closed = set()
        closed.add(problem.initial)

    while len(fringe) > 0:
        pv = fringe.peek_value()

        if bv < pv:
            break

        node = fringe.pop()

        if problem.goal_test(node, problem.goal):
            yield SolutionNode(node, problem.goal)

        if problem.node_value(node) < bv:
            b = node
            bv = problem.node_value(node)
            fringe.update_cost_limit(bv)

        if depth_limit == float('inf') or node.depth() < depth_limit:
            for s in problem.successors(node):
                if not graph:
                    fringe.push(s)
                elif s not in closed:
                    fringe.push(s)
                    closed.add(s)

    yield SolutionNode(b, problem.goal)


def hill_climbing(problem, random_restarts=0, max_sideways=0, graph=True):
    """
    Probably the simplest optimization approach. It expands the list of
    neighbors and chooses the best neighbor (steepest descent hill climbing).

    Default configuration should yield similar behavior to
    :func:`local_beam_search` when it has a width of 1, but doesn't need to
    maintain alternatives, so might use slightly less memory (just stores the
    best node instead of limited length priority queue).

    If graph is true (the default), then a closed list is maintained.  This is
    imporant for search spaces with platues because it keeps the algorithm from
    reexpanding neighbors with the same value and getting stuck in a loop.

    If random_restarts > 0, then search is restarted multiple times. This can
    be useful for getting out of local minimums.

    The problem.goal_test function can be used to terminate search early if a
    good enough solution has been found. If goal_test(node) return True, then
    search is immediately terminated and the node is returned.

    :param problem: The problem to solve.
    :type problem: :class:`py_search.base.Problem`
    :param random_restarts: The number of times to restart search. The
        initial state is used for the first search and subsequent starts begin
        at a random state.
    :type random_restarts: int
    :param max_sideways: Specifies the max number of contiguous sideways moves.
    :type max_sideways: int
    :param graph: Whether to use graph search (no duplicates) or tree search
        (duplicates)
    :type graph: Boolean
    """
    b = problem.initial
    bv = problem.node_value(b)

    if problem.goal_test(b, problem.goal):
        yield SolutionNode(b, problem.goal)

    if graph:
        closed = set()
        closed.add(problem.initial)

    c = b
    cv = bv

    while random_restarts >= 0:
        found_better = True
        sideways_moves = 0
        while found_better and sideways_moves <= max_sideways:
            found_better = False
            prev_cost = cv
            for s in problem.successors(c):
                if graph and s in closed:
                    continue
                elif graph:
                    closed.add(s)
                sv = problem.node_value(s)
                if sv <= bv:
                    b = s
                    bv = sv
                    if problem.goal_test(b, problem.goal):
                        yield SolutionNode(b, problem.goal)
                if sv <= cv:
                    c = s
                    cv = sv
                    found_better = True
            if cv == prev_cost:
                sideways_moves += 1
            else:
                sideways_moves = 0

        random_restarts -= 1
        if random_restarts >= 0:
            c = problem.random_node()
            while graph and c in closed:
                c = problem.random_node()
            cv = problem.node_value(c)

            if graph:
                closed.add(c)
            if cv <= bv:
                b = c
                bv = cv
                if problem.goal_test(b, problem.goal):
                    yield SolutionNode(b, problem.goal)

    yield SolutionNode(b, problem.goal)


def local_beam_search(problem, beam_width=1, max_sideways=0, graph=True):
    """
    A variant of :func:`py_search.informed_search.beam_search` that can be
    applied to local search problems.  When the beam width of 1 this approach
    yields behavior similar to :func:`hill_climbing`.

    The problem.goal_test function can be used to terminate search early if a
    good enough solution has been found. If goal_test(node) return True, then
    search is immediately terminated and the node is returned.

    :param problem: The problem to solve.
    :type problem: :class:`py_search.base.Problem`
    :param beam_width: The size of the search beam.
    :type beam_width: int
    :param max_sideways: Specifies the max number of contiguous sideways moves.
    :type max_sideways: int
    :param graph: Whether to use graph search (no duplicates) or tree
        search (duplicates)
    :type graph: Boolean
    """
    b = None
    bv = float('inf')
    sideways_moves = 0

    fringe = PriorityQueue(node_value=problem.node_value)
    fringe.push(problem.initial)

    while len(fringe) < beam_width:
        fringe.push(problem.random_node())

    if graph:
        closed = set()
        closed.add(problem.initial)

    while len(fringe) > 0 and sideways_moves <= max_sideways:
        pv = fringe.peek_value()

        if bv < pv:
            yield SolutionNode(b, problem.goal)

        if pv == bv:
            sideways_moves += 1
        else:
            sideways_moves = 0

        parents = []
        while len(fringe) > 0 and len(parents) < beam_width:
            parent = fringe.pop()
            parents.append(parent)
        fringe.clear()

        b = parents[0]
        bv = pv

        for node in parents:
            if problem.goal_test(node, problem.goal):
                yield SolutionNode(node, problem.goal)

            for s in problem.successors(node):
                if not graph:
                    fringe.push(s)
                elif s not in closed:
                    fringe.push(s)
                    closed.add(s)

    yield SolutionNode(b, problem.goal)


def simulated_annealing(problem, temp_factor=0.95, temp_length=None,
                        initial_temp=None, init_prob=0.4, min_accept=0.02,
                        min_change=1e-6, limit=float('inf')):
    """
    A more complicated optimization technique. At each iteration a random
    successor is expanded if it is better than the current node. If the random
    successor is not better than the current node, then it is expanded with
    some probability based on the temperature.

    Used the formulation of simulated annealing found in:
        Johnson, D. S., Aragon, C. R., McGeoch, L. A., & Schevon, C. (1989).
        Optimization by simulated annealing: an experimental evaluation; part
        I, graph partitioning. Operations research, 37(6), 865-892.

    Also, the problem.goal_test function can be used to terminate search early
    if a good enough solution has been found. If goal_test(node) return True,
    then search is immediately terminated and the node is returned.

    :param problem: The problem to solve.
    :type problem: :class:`py_search.base.Problem`
    :param temp_factor: The factor for geometric cooling, a value between 0 and
        1, but usually very close to 1.
    :type temp_factor: float
    :param temp_length: The number of nodes to expand at each temperature. If
        set to `None` (the default) then it is automatically chosen to be equal
        to the length of the successors list.
    :type temp_length: int
    :param initial_temp: The initial temperature for the annealing. The number
        is objective function specific. If set to None (the default), then
        a semi-random walk is used to select an initial
        temperature that will yield approx. init_prob acceptance rate for
        worse states.
    :type initial_temp: float or None
    :param min_accept: The fraction of states that must be accepted in
        temp_length iterations (taken from a single temperature) to not be
        frozen.  Every time this is not exceeded, the frozen counter is
        incremented until it hits 5. If a better state is found, then the
        frozen counter is reset to 0.
    :type min_accept: float between 0 and 1
    :param min_change: The amount of change that must be achieved in
        temp_length iterations (taken from a single temperature) to not be
        frozen.  Everytime this is not exceeded, the frozen counter is
        incremented until it hits 5. If a better state is found, then the
        frozen counter is reset to 0.
    :type min_change: float
    :param limit: The maximum number of iterations (random neighbors) to expand
        before stopping.
    :type limit: float
    """
    T = initial_temp
    b = problem.initial
    bv = problem.node_value(b)

    if problem.goal_test(b, problem.goal):
        yield SolutionNode(b, problem.goal)

    c = b
    cv = bv

    frozen = 0
    iterations = 0

    if temp_length is None:
        temp_length = len(list(problem.successors(c)))
        print("Temp length set equal to number of initial neighbors (%i)" %
              temp_length)

    if T is None:
        delta_sum = 0
        delta_count = 0

    while frozen < 5:
        acceptances = 0
        changes = 0

        for i in range(temp_length):
            iterations += 1
            s = problem.random_successor(c)
            sv = problem.node_value(s)

            if sv < bv:
                b = s
                bv = sv
                frozen = 0

            if iterations >= limit:
                yield SolutionNode(b, problem.goal)

            delta_e = sv - cv
            if T is None and delta_e > 0:
                delta_sum += delta_e
                delta_count += 1
            if ((delta_e <= 0 or (T is None and random() < 0.5) or
                 (T is not None and T > 1e-2 and random() < exp(-delta_e/T)))):
                acceptances += 1
                changes += delta_e
                c = s
                cv = sv

                if problem.goal_test(c, problem.goal):
                    yield SolutionNode(c, problem.goal)

        if T is None:
            if delta_count > 100:
                avg_delta = delta_sum / delta_count
                T = -avg_delta / log(init_prob)
                print("Initial temperature set to:",
                      "%0.3f (based on %i samples)" % (T, delta_count))
        else:
            T = temp_factor * T

        if (((acceptances / temp_length) < min_accept or
             abs(changes) < min_change)):
            frozen += 1

    yield SolutionNode(b, problem.goal)

# Utilities

In [40]:
"""
Utilities for the py_search library.
"""
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import division

from tabulate import tabulate
from random import uniform
from functools import wraps
from functools import partial
import timeit

# from py_search.base import AnnotatedProblem


def weighted_choice(choices):
    """
    Given a list of weighted choices, choose one.  Choices are a list of
    (weight, element) pairs.
    """
    total = sum(w for w, c in choices)
    r = uniform(0, total)
    upto = 0
    for w, c in choices:  # pragma: no branch
        if upto + w >= r:
            return c
        upto += w


def compare_searches(problems, searches):
    """
    A function for comparing different search algorithms on different problems.

    :param problems: problems to solve.
    :type problems: an iterator of problems (usually a list)
    :param searches: search algorithms to use.
    :type searches: an iterator of search functions (usually a list)
    """
    table = []

    for problem in problems:
        for search in searches:
            annotated_problem = AnnotatedProblem(problem)
            start_time = timeit.default_timer()

            try:
                sol = next(search(annotated_problem))
                elapsed = timeit.default_timer() - start_time
                cost = sol.cost()
            except StopIteration:
                elapsed = timeit.default_timer() - start_time
                cost = 'Failed'
            
            table.append([#problem.__class__.__name__, 
                          search.__name__,
                          annotated_problem.goal_tests,
                          annotated_problem.nodes_expanded,
                          annotated_problem.nodes_evaluated, "%0.3f" % cost if
                          isinstance(cost, float) else cost,
                          "%0.4f" % elapsed if isinstance(elapsed, float) else
                          elapsed, sol.state_node.state])

    print(tabulate(table, headers=['Search Alg', 'Goal Tests',
                                   'Nodes Expanded', 'Nodes Evaluated',
                                   'Solution Cost', 'Runtime', 'Solution'],
                   tablefmt="simple"))


def timefun(f):
    """
    A decorator function for calling Timer with autorange on the provided
    function.
    """

    @wraps(f)
    def wrapper(*args, **kwds):
        try:
            result = timeit.Timer(partial(f, *args, **kwds)).autorange()
        except Exception:
            result = [10, timeit.Timer(partial(f, *args, **kwds)).timeit(10)]
        a = [a for a in args]
        a += ["%s=%s" % (k, kwds[k]) for k in kwds]
        print("Timing %s%s: %0.7f (num runs=%i)" % (f.__name__, tuple(a),
                                                    result[1], result[0]))
        return f(*args, **kwds)

    return wrapper

# Genetic algorithm class

In [41]:
import random

class GeneticAlgorithm:
    def __init__(self, problem, population_size=10, crossover_rate=0.9, mutation_rate=0.01):
        self.problem = problem
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = # ... which problem method create a random node
            population.append(individual)
        return population

    def evaluate_fitness(self, individual):
        node_value = problem.node_value(individual)
        return node_value

    def pmx_crossover(self, parent1, parent2):
        """Perform Partial Mapped Crossover (PMX) on two parent chromosomes."""
        # Choose two random crossover points checkpoint1 and checkpoint2 to create a crossover segment (in range of 0 and parent length)
            ...    
        # Create copies of parent1 and parent2 to obtain offspring1 and offspring2.
            ...
        # Make sure checkpoin1 < checkpoin2
            ...
        # Copy the values from parent1 to offspring2 and from parent2 to offspring1 for the crossover segment (between checkpoint1 and checkpoint2)
            ...
        # Create a mapping that aligns the parents' values within the crossover segment
            ...
        # Update offspring1 and offspring2 by replacing duplicated values outside the crossover segment with mapped values
            # Use visited_indices set to avoid an infinite loop
            ...
        
        return offspring1, offspring2

    def crossover(self, parent1, parent2):
        #**************Performe partial mapped crossover************
        offspring1, offspring2 = self.pmx_crossover(parent1.state, parent2.state)
        #Turn the lists into nodes
        child1 = Node(offspring1, node_cost=cost(offspring1, problem.initial.extra[0]),extra=problem.initial.extra)
        child2 = Node(offspring2, node_cost=cost(offspring2, problem.initial.extra[0]),extra=problem.initial.extra)
        return child1, child2

    def mutate(self, individual):
        mutated_individual = individual
        if random.random() < self.mutation_rate:
            mutated_individual = # ... Which problem's method allows you to perform mutation (swapping) ?
        return mutated_individual

    def evolve_population(self, population):
        # ... Select two parents (individuals) from the population
        if random.random() < self.crossover_rate:
            child1, child2 = self.crossover(parent1, parent2)
        else:
            #...What should you do here ?
        # ... Mutate child1
        # ... Mutate child2
        population.extend([child1,child2])
        return population

    def select_individuals(self, population, num_parents):
        parent1, parent2 = random.choices(population, k=num_parents)
        return parent1, parent2

    def run(self, num_generations):
        population = # ... Generate the initial population
        print(f"The best solution in the initial population : {}")# Print the best individual of the initial population
        for i in range(num_generations):
            print(f"Generation {i}")
            population = #... Evolve !
            
            best_generation = # ... Get the best individual of this generation
            print(f"Generation {i}, Best Individual:{best_generation}")
        best_individual = # ... Get the final best indivdual 
        solution_node = # ... Create a solution node out of best_individual
        yield solution_node

SyntaxError: invalid syntax (3064818598.py, line 13)

# Assignment Problem

In [35]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import division
from itertools import combinations
from random import normalvariate
from random import shuffle
from random import randint

from munkres import Munkres

def random_matrix(n):
    """
    Generates an a list of list of floats (representing an n x n matrix) where
    the values have mean 0 and std 1.

    This is used as cost matrix for an assignment problem.
    """
    cost_matrix = [
    [9, 2, 7, 8, 1, 3, 9, 5],
    [6, 4, 3, 7, 3, 4, 1, 9],
    [5, 8, 1, 8, 2, 9, 7, 9],
    [7, 6, 9, 4, 5, 2, 5, 7],
    [2, 2, 5, 3, 7, 7, 5, 6],
    [1, 4, 2, 1, 2, 6, 3, 7],
    [6, 8, 4, 6, 1, 5, 7, 8],
    [8, 2, 3, 5, 5, 4, 3, 7]
    ]

    return cost_matrix


class AssignmentProblem(Problem):
    """
    A tree search version of the assignment problem. Starts with an initially
    empty assignment and then incrementally builds the assignment up adding one
    assignment per expansion.
    """

    def min_cost_heuristic(self, node):
        """
        A huristic specifying the minimum cost that could be achieved for
        unassigned rows.
        """
        node.state
        costs, unassigned = node.extra

        empty_rows = [i for i, v in enumerate(node.state) if v is None]

        min_possible = 0
        for r in empty_rows:
            sub_c = [v for i, v in enumerate(costs[r]) if i in unassigned]
            min_possible += min(sub_c)

        return min_possible

    def node_value(self, node):
        """
        The value of a node is the combination of the node cost and the
        min_cost heuristic
        """
        return node.cost() + self.min_cost_heuristic(node)

    def successors(self, node):
        """
        An iterator that yields the sucessors of the provided node.
        """
        state = node.state
        costs, unassigned = node.extra

        for i, v in enumerate(state):
            if v is None:
                for u in unassigned:
                    new_state = tuple([u if i == j else k
                                       for j, k in enumerate(state)])
                    new_unassigned = tuple([k for k in unassigned if k != u])

                    c = node.cost() + costs[i][u]
                    yield Node(new_state, node, (i, u), c,
                               extra=(costs, new_unassigned))

    def goal_test(self, node, goal):
        """
        A test of whether a complete assignment has been reached.
        """
        state = node.state
        return None not in state


class LocalAssignmentProblem(Problem):
    """
    This class represents a local search version of the assignment problem.
    I.e., a random state is generated to start the search and then neighbors of
    the state can be expanded in order to reduce the solution cost.
    """

    def node_value(self, node):
        """
        Returns a lower bound on the solution cost reachable from the given
        node (or its children)
        """
        return float('-inf')

    def random_successor(self, node):
        """
        A function that returns a random successor of the current node. This is
        used by the simulated annealing function, so it doesn't have to expand
        all successors.

        A successor is generated by randomly flipping a pair of row to column
        assignments.
        """
        costs = node.extra[0]
        p = [0, 0]
        p[0] = randint(0, len(node.state)-1)
        p[1] = p[0]
        while p[0] == p[1]:
            p[1] = randint(0, len(node.state)-1)
            
        new_cost = node.cost()
        new_cost -= costs[p[0]][node.state[p[0]]]
        new_cost -= costs[p[1]][node.state[p[1]]]
        new_cost += costs[p[0]][node.state[p[1]]]
        new_cost += costs[p[1]][node.state[p[0]]]

        state = list(node.state)
        state[p[0]], state[p[1]] = state[p[1]], state[p[0]]

        return Node(tuple(state), node, p, new_cost, extra=node.extra)

    def successors(self, node):
        """
        Generates successor states by flipping each pair of row to column
        assignments.
        """
        costs = node.extra[0]

        for p in combinations(node.state, 2):
            new_cost = node.cost()
            new_cost -= costs[p[0]][node.state[p[0]]]
            new_cost -= costs[p[1]][node.state[p[1]]]
            new_cost += costs[p[0]][node.state[p[1]]]
            new_cost += costs[p[1]][node.state[p[0]]]

            state = list(node.state)
            temp = state[p[0]]
            state[p[0]] = state[p[1]]
            state[p[1]] = temp

            yield Node(tuple(state), node, p, new_cost, extra=node.extra)

    def random_node(self):
        """
        Generates a node that has a random assignment.
        """
        state = random_assignment(len(self.initial.state))
        return Node(state, node_cost=cost(state, self.initial.extra[0]),
                    extra=self.initial.extra)

    def goal_test(self, node, goal):
        return False


def random_assignment(n):
    """
    Returns a random valid assignment for an n x n matrix
    """
    state = list(range(n))
    shuffle(state)
    return tuple(state)


def cost(assignment, costs):
    """
    Given an assignemnt and a cost matrix, returns the cost of the
    assignment.
    """
    cost = 0.0
    for row, col in enumerate(assignment):
        cost += costs[row][col]
    return cost


def print_matrix(m):
    """
    Print a matrix
    """
    for row in m:
        print("\t".join(["%0.2f" % v for v in row]))





####################################################
Randomly generated square cost matrix (8 x 8)
####################################################
9.00	2.00	7.00	8.00	1.00	3.00	9.00	5.00
6.00	4.00	3.00	7.00	3.00	4.00	1.00	9.00
5.00	8.00	1.00	8.00	2.00	9.00	7.00	9.00
7.00	6.00	9.00	4.00	5.00	2.00	5.00	7.00
2.00	2.00	5.00	3.00	7.00	7.00	5.00	6.00
1.00	4.00	2.00	1.00	2.00	6.00	3.00	7.00
6.00	8.00	4.00	6.00	1.00	5.00	7.00	8.00
8.00	2.00	3.00	5.00	5.00	4.00	3.00	7.00

####################################################
Optimial solution using Munkres/Hungarian Algorithm
####################################################
Munkres Solution:
(7, 6, 2, 5, 0, 3, 4, 1)
Munkres Cost:
15.0

######################################
Local Search / Optimization Techniques
######################################
Initial Assignment (randomly generated):
(3, 1, 2, 5, 6, 4, 7, 0)
Initial Assignment Cost:
38.0

Initial Assignment Goal:
State: None, Cost: 0


###########################################

AttributeError: 'tuple' object has no attribute 'copy'

# Main

In [ ]:
if __name__ == "__main__":

    n = 8
    costs = random_matrix(n)

    print()
    print("####################################################")
    print("Randomly generated square cost matrix (%i x %i)" % (n, n))
    print("####################################################")
    print_matrix(costs)
    
    
    print()
    print("####################################################")
    print("Optimial solution using Munkres/Hungarian Algorithm")
    print("####################################################")

    m = Munkres()
    indices = m.compute(costs)
    best = tuple([v[1] for v in indices])
    print("Munkres Solution:")
    print(best)
    print("Munkres Cost:")
    print(cost(best, costs))
    print()

    print("######################################")
    print("Local Search / Optimization Techniques")
    print("######################################")

    initial = random_assignment(n)
    problem = LocalAssignmentProblem(initial,
                                     initial_cost=cost(initial, costs),
                                     extra=(costs,))
    print("Initial Assignment (randomly generated):")
    print(initial)
    print("Initial Assignment Cost:")
    print(problem.initial.cost())
    print()
    
    
    print("###############################################################################")
    print("****************************Genetic Algorithm**********************************")
    print("###############################################################################")
    
    # ... Create your genetic algorithm object and run it here.
    

    def local_beam_width2(problem):
        return local_beam_search(problem, beam_width=2)

    def greedy_annealing(problem):
        num_neighbors = (n * (n-1)) // 2
        return simulated_annealing(problem, initial_temp=0,
                                   temp_length=num_neighbors)

    def depth_limited_branch_and_bound(problem):
        return branch_and_bound(problem, depth_limit=100)
    

    compare_searches(problems=[problem],
                     searches=[depth_limited_branch_and_bound,
                               hill_climbing,
                               local_beam_width2,
                               greedy_annealing,
                              ])
